In [ ]:
#예제 10.1 평균에 의한 배경 영상
#평균에 의한 배경영상
#동영상 재생시 매 frame을 더한 뒤 frame의 수로 나눈다.
#이동하는 물체는 지속적으로 움직여 frame에서 사라지거나 값이 계속 바뀌므로 결국 움직이지 않는 background만 남게 된다.
import cv2
import numpy as np
#1
cap = cv2.VideoCapture('0205.png\\vtest.avi')   #비디오 객체 cap 생성
if (not cap.isOpened()):
    print('Error opening video')

height, width = (int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)),
                int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))) #프레임 크기 읽기

acc_gray = np.zeros(shape= (height, width), dtype= np.float32)  #그레이스케일, 컬러 프레임 누적을 위한 acc_gray, acc_bgr 생성
acc_bgr = np.zeros(shape= (height, width, 3), dtype= np.float32)
t = 0

#2
while True:
    ret, frame = cap.read()
    if not ret:
        break
    t += 1
    print('t=', t)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#2-1
    cv2.accumulate(gray, acc_gray)  #gray 영상 누적
    avg_gray = acc_gray / t #avg_gray: 매 frame을 gray로 변환후 더한 뒤 frame의 수로 나눈다
    dst_gray = cv2.convertScaleAbs(avg_gray) #절대값 계산후 8bit로서 표현
#2-2
    cv2.accumulate(frame, acc_bgr)  #color 영상 누적
    avg_bgr = acc_bgr / t #avg_bgr: 매 frame을 더한 뒤 frame의 수로 나눈다.
    dst_bgr = cv2.convertScaleAbs(avg_bgr) #절대값 계산후 8bit로서 표현

    cv2.imshow('frame', frame)
    cv2.imshow('dst_gray', dst_gray)
    cv2.imshow('dst_bgr', dst_bgr)
    key = cv2.waitKey(20)
    if key == 27:
        break
#3
if cap.isOpened(): cap.release()
cv2.imwrite('./data/avg_gray.png', dst_gray)
cv2.imwrite('./data/avg_bgr.png', dst_bgr)
cv2.destroyAllWindows()

In [ ]:
#예제 10.2 배경 차영상 이동물체 검출
import cv2
import numpy as np

#1
cap = cv2.VideoCapture('0205.png\\vtest.avi')   #비디오 객체 cap 생성
if (not cap.isOpened()):
    print('Error opening video')

height, width = (int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)),
                int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))) #프레임 크기 읽기

TH = 40                             #binary threshold
AREA_TH = 80                        #area   threshold
bkg_gray = cv2.imread('./data/avg_gray.png', cv2.IMREAD_GRAYSCALE)
bkg_bgr = cv2.imread('./data/avg_bgr.png')

mode = cv2.RETR_EXTERNAL
method = cv2.CHAIN_APPROX_SIMPLE

#2
t = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break
    t += 1
    print('t=', t)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

#2-1
    diff_gray = cv2.absdiff(gray, bkg_gray)
##  ret, bImage = cv2.threshold(diff_gray, TH, 255,
##                              cv2.THRESH_BINARY)    

#2-2
    diff_bgr = cv2.absdiff(frame, bkg_bgr)
    db, dg, dr = cv2.split(diff_bgr)
    ret, bb = cv2.threshold(db, TH, 255, cv2.THRESH_BINARY)
    ret, bg = cv2.threshold(dg, TH, 255, cv2.THRESH_BINARY)
    ret, br = cv2.threshold(dr, TH, 255, cv2.THRESH_BINARY)
    
    bImage = cv2.bitwise_or(bb, bg)
    bImage = cv2.bitwise_or(bb, bImage)

    bImage = cv2.erode(bImage, None, 5)
    bImage = cv2.dilate(bImage, None, 5)
    bImage = cv2.erode(bImage, None, 7)
        
#2-3
    image, contours, hierarchy = cv2.findContours(bImage, mode, method)
    cv2.drawContours(frame, contours, -1, (255, 0, 0), 1)
    for i, cnt in enumerate(contours):
        area = cv2.contourArea(cnt)
        if area > AREA_TH:
            x, y, width, height = cv2.boundingRect(cnt)
            cv2.rectangle(frame, (x, y), (x + width, y + height),
                            (0, 0, 255), 2)
    
    cv2.imshow('frame', frame)
    cv2.imshow('bImage', bImage)
    cv2.imshow('diff_gray', diff_gray)
    cv2.imshow('diff_bgr', diff_bgr)
    key = cv2.waitKey(25)
    if key == 27:
        break
#3
if cap.isOpened():
    cap.release()
cv2.destroyAllWindows()

In [ ]:
#예제 10.2 배경 차영상 이동물체 검출(다른 코드)
import cv2
import numpy as np

#1
cap = cv2.VideoCapture('0205.png\\vtest.avi')

if not cap.isOpened():
    print('Video open failed!')
    sys.exit()

# 배경 영상 등록
ret, back = cap.read()

if not ret:
    print('Background image registration failed!')
    sys.exit()
    
# 연산 속도를 높이기 위해 그레이스케일 영상으로 변환
back = cv2.cvtColor(back, cv2.COLOR_BGR2GRAY)

# 가우시안 블러로 노이즈 제거 (모폴로지, 열기, 닫기 연산도 가능)
back = cv2.GaussianBlur(back, (0, 0), 1.0)

# 비디오 매 프레임 처리
while True:
    ret, frame = cap.read()
    
    if not ret:
        break
    
    # 현재 프레임 영상 그레이스케일 변환
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # 노이즈 제거
    gray = cv2.GaussianBlur(gray, (0, 0), 1.0)
    
    # 차영상 구하기 $ 이진화
    # absdiff는 차 영상에 절대값
    diff = cv2.absdiff(gray, back)
    # 차이가 30이상 255(흰색), 30보다 작으면 0(검정색)
    _, diff = cv2.threshold(diff, 30, 255, cv2.THRESH_BINARY)
    
    # 레이브링을 이용하여 바운딩 박스 표시
    cnt, _, stats, _ = cv2.connectedComponentsWithStats(diff)
    
    for i in range(1, cnt):
        x, y, w, h, s = stats[i]
        
        if s < 100:
            continue
            
        cv2.rectangle(frame, (x, y, w, h), (0, 0, 255), 2)
    
    cv2.imshow('frame', frame)
    cv2.imshow('diff', diff)

    if cv2.waitKey(30) == 27:
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
#예제 10.3 이동평균 moving average 배경 차영상
import cv2
import numpy as np

#1
cap = cv2.VideoCapture('0205.png\\vtest.avi')   #비디오 객체 cap 생성
if (not cap.isOpened()):
    print('Error opening video')

height, width = (int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)),
                int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))) #프레임 크기 읽기

TH = 40                             #binary threshold
AREA_TH = 80                        #area   threshold
acc_bgr = np.zeros(shape= (height, width, 3), dtype= np.float32)

mode = cv2.RETR_EXTERNAL
method = cv2.CHAIN_APPROX_SIMPLE

#2
t = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break
    t += 1
    print('t=', t)
    blur = cv2.GaussianBlur(frame, (5, 5), 0.0)
#2-1
    if t < 50:
        cv2.accumulate(blur, acc_bgr)
        continue
    elif t == 50:
        bkg_bgr = acc_bgr / t
#2-2: t >= 50
##    diff_bgr = cv2.absdiff(np.float32(blur).
#                            bkg_bgr).astype(np.uint8)
    diff_bgr = np.uint8(cv2.absdiff(np.float32(blur), bkg_bgr))
    db, dg, dr = cv2.split(diff_bgr)
    ret, bb = cv2.threshold(db, TH, 255, cv2.THRESH_BINARY)
    ret, bg = cv2.threshold(dg, TH, 255, cv2.THRESH_BINARY)
    ret, br = cv2.threshold(dr, TH, 255, cv2.THRESH_BINARY)
    bImage = cv2.bitwise_or(bb, bg)
    bImage = cv2.bitwise_or(br, bImage)
    bImage = cv2.erode(bImage, None, 5)
    bImage = cv2.dilate(bImage, None, 5)
    bImage = cv2.erode(bImage, None, 7)
    
    cv2.imshow('bImage', bImage)
    msk = bImage.copy()
    contours, hierarchy = cv2.findContours(bImage, mode, method)
    cv2.drawContours(frame, contours, -1, (255, 0, 0), 1) #cv2.drawContontours()를 통하여 윤곽선 검출
    for i, cnt in enumerate(contours):
        area = cv2.contourArea(cnt) #cv2.contourArea()를 통하여 Moving Object의 크기 추출후 특정 크기 이상의 물체만 검출
        if area > AREA_TH:
            x, y, width, height = cv2.boundingRect(cnt)
            cv2.rectangle(frame, (x, y), (x + width, y + height),
                            (0, 0, 255), 2)
            cv2.rectangle(msk, (x, y), (x + width, y + height),
                            255, -1)
    
#2-3
    msk = cv2.bitwise_not(msk)  #이동물체 영역은 0, 배경은 255로 반전
    cv2.accumulateWeighted(blur, bkg_bgr, alpha= 0.1, mask = msk) #이동 평균 영상
#Mask는 배경이 255인 곳만을 가르키고 있으므로 위에서 dst는 배경에만 영향을 받아서 계속되어 Update가 된다는 것을 알 수 있다.
    cv2.imshow('frame', frame)
    cv2.imshow('bkg_bgr', np.uint8(bkg_bgr))
    cv2.imshow('diff_bgr', diff_bgr)
    key = cv2.waitKey(25)
    if key == 27:
        break
#3
if cap.isOpened():
    cap.release()
cv2.destroyAllWindows()
#장점: 현재 영상에서 추가로 다른 Object(EX) 가로등, 신호등)가 설치가 되어도 지속적인 움직임이 없으면 Background로 포함시킬 수 있다는 것

In [ ]:
#예제 10.4 BackgroundSubtractor 배경과 전경 분할
import cv2
import numpy as np

cap = cv2.VideoCapture('0205.png\\vtest.avi')   #비디오 객체 cap 생성
if (not cap.isOpened()):
    print('Error opening video')

height, width = (int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)),
                int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))) #프레임 크기 읽기
#1
bgMog1 = cv2.createBackgroundSubtractorMOG2() #마할라노비스 거리 제곱 사용
bgMog2 = cv2.createBackgroundSubtractorMOG2(varThreshold= 25,
                                             detectShadows= False)
bgknn1 = cv2.createBackgroundSubtractorKNN()
bgknn2 = cv2.createBackgroundSubtractorKNN(dist2Threshold= 1000,
                                            detectShadows= False)

#2 이진영상 bImage에 모폴로지 연산, 윤곽선 검출
AREA_TH  = 80   #area threshold
def findObjectAndDraw(bImage, src):
    res = src.copy()
    bImage = cv2.erode(bImage, None, 5)
    bImage = cv2.dilate(bImage, None, 5)
    bImage = cv2.erode(bImage, None, 7)
    contours, _ = cv2.findContours(bImage,
                        cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cv2.drawContours(src, contours, -1, (255, 0, 0), 1)
    for i, cnt in enumerate(contours):
        area = cv2.contourArea(cnt)
        if area > AREA_TH:
            x, y, width, height = cv2.boundingRect(cnt)
            cv2.rectangle(res, (x, y), (x + width, y + height),
                            (0, 0, 255), 2)
    return res

t = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break
    t+=1
    print('t=', t)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(frame, (5, 5), 0.0)

#3
    bImage1 = bgMog1.apply(blur)
    bImage2 = bgMog2.apply(blur)
    bImage3 = bgknn1.apply(blur)
    bImage4 = bgknn2.apply(blur)
    dst1 = findObjectAndDraw(bImage1, frame)
    dst2 = findObjectAndDraw(bImage1, frame)
    dst3 = findObjectAndDraw(bImage1, frame)
    dst4 = findObjectAndDraw(bImage1, frame)
    
##      if t == 50:
    cv2.imshow('bImage1', bImage1)
    cv2.imshow('bgMog1', dst1)
    cv2.imshow('bImage2', bImage2)
    cv2.imshow('bgMog2', dst2)
    cv2.imshow('bImage3', bImage3)
    cv2.imshow('bgKnn1', dst3)
    cv2.imshow('bImage4', bImage4)
    cv2.imshow('bgKnn2', dst4)
    key = cv2.waitKey(25)
    if key == 27:
        break
if cap.isOpened():
    cap.release()
cv2.destroyAllWindows()

In [ ]:
#예제 10.7 cv2.meanShift(), cv2.Camshift() 추적
import cv2
import numpy as np

#1 마우스 이벤트를 통해 roi 영역 설정
roi = None
drag_start = None
mouse_status = 0
tracking_start = False
def onMouse(event, x, y, flags, param = None):
    global roi
    global drag_start
    global mouse_status
    global tracking_start
    if event == cv2.EVENT_LBUTTONDOWN:
        drag_start = (x, y)
        mouse_status = 1
        tracking_start = False
    elif event == cv2.EVENT_MOUSEMOVE:
        if flags == cv2.EVENT_FLAG_LBUTTON:
            xmin = min(x, drag_start[0])
            ymin = min(y, drag_start[1])
            xmax = max(x, drag_start[0])
            ymax = max(y, drag_start[1])
            roi = (xmin, ymin, xmax, ymax)
            mouse_status = 2    #dragging
    elif event == cv2.EVENT_LBUTTONUP:
        mouse_status = 3        #complete

#2
cv2.namedWindow('tracking') #윈도우 생성
cv2.setMouseCallback('tracking', onMouse) #onMouse를 마우스 이벤트 콜백 함수로 설정

cap = cv2.VideoCapture(0)
if (not cap.isOpened()):
    print('Error opening video')
height, width = (int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)),
                 int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))) #프레임 크기 읽기
roi_mask = np.zeros((height, width), dtype= np.uint8) #roi_mask 0으로 초기화
term_crit = (cv2.TERM_CRITERIA_MAX_ITER + cv2.TERM_CRITERIA_EPS,    #종료조건 설정
             10, 1)

#3
t = 0
while True:
    ret, frame = cap.read()
    if not ret: break
    t += 1
    print('t=', t)
#3-1
    frame2 = frame.copy()   #CamShift 추적 결과 표시를 위한 복사 영상
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, (0., 60., 32.), (180., 255., 255.)) #어둡고 흐릿한 영역 제외 #H[0, 180], S[60, 255], V[32, 255] 범위 포함 255, 아닌 영역은 0인 이진영상
##      cv2.imshow('mask', mask)
#3-2 마우스 드래깅시에 frame에 파란 사각형 표시
    if mouse_status==2:
        x1, y1, x2, y2 = roi
        cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2) 
#3-3 
    if mouse_status == 3:
        print('initialize....')
        mouse_status = 0
        x1, y1, x2, y2 = roi
        mask_roi = mask[y1:y2, x1:x2]
        hsv_roi = hsv[y1:y2, x1:x2]         #roi 영역 저장

        hist_roi = cv2.calcHist([hsv_roi], [0], mask_roi,   #hsv_roi의 H채널 히스토그램 계싼
                                [16], [0, 180])
        cv2.normalize(hist_roi, hist_roi, 0, 255, cv2.NORM_MINMAX)  #히스토그램 빈도수 정규화
        track_window1 = (x1, y1, x2 - x1, y2 - y1)  #meanshift
        track_window2 = (x1, y1, x2 - x1, y2 - y1)  #camshift
        tracking_start = True   #추적 시작
#3-4
    if tracking_start:
        backP = cv2.calcBackProject([hsv], [0], hist_roi, [0, 180], 1) #H채널 히스토그램 역투영한 backP생성
        backP &= mask   #AND연산 -> 역투영에서 어둡고 흐릿한 영역 제외
        cv2.imshow('backP', backP)

#3-5: meanShift tracking -> 역투영을 이용해서 관심영역 물체 추적
        ret, track_window1 = cv2.meanShift(backP, track_window1,
                                            term_crit)
        x, y, w, h = track_window1
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)

#3-6: camShift tracking -> 역투영을 통해 관심영역 물체 추적
        track_box, track_window2 = cv2.CamShift(backP,
                                                track_window2,
                                                term_crit)
        x, y, w, h = track_window2
        cv2.rectangle(frame2, (x, y), (x + w, y + h), (0, 255, 0), 2)   #초록색 사각형
        cv2.ellipse(frame2, track_box, (0, 255, 255), 2)    #노란색 타원
        pts = cv2.boxPoints(track_box)
        pts = np.int0(pts)      #np.int32
        dst = cv2.polylines(frame2,[pts],True, (0, 0, 255), 2)  #빨간색 회전 사각형
##          cv2.imshow('tracking', frame)
#           cv2.imshow('CamShift tracking', frame2)
#           cv2.waitKey(0)
    cv2.imshow('tracking', frame)                 # meanShift        
    cv2.imshow('CamShift tracking', frame2)       # CamShift
    key = cv2.waitKey(25)
    if key == 27:
        break
if cap.isOpened(): cap.release();
cv2.destroyAllWindows()